In [57]:
import subprocess
import requests
import urllib.request
import zipfile
import os
import shutil
import geopandas as gpd
from bs4 import BeautifulSoup
import urllib.request
from os import listdir
from os.path import isfile, join
import pandas as pd
import ee
import os
import glob
from zipfile import ZipFile
import datetime as dt

ee.Initialize()


# PART 1 DOWNLOAD DATA


In [ ]:
ex = ['.shp', '.prj', '.dbf', '.shx']
file_paths = []
i=1

def zipshape(directory,export):
    for (root, directories, files) in os.walk(directory):
        for filename in files:
            if filename.endswith('.shp'):
                file_paths=[]
                filebase = filename.split('.')[0]
                try:
                    for things in ex:
                        if os.path.exists(os.path.join(root, filebase+things)):
                            filepath = os.path.join(root, filebase+things)
                            #print(filepath)
                            file_paths.append(filepath)
                    os.chdir(export)
                    if not os.path.exists(filebase+'.zip') and int(len(file_paths))==int(4):
                        with ZipFile(filebase+'.zip','w') as zip:
                            print("Creating zipped folder "+str(filebase)+'.zip'+' at '+str(export))
                            # writing each file one by one
                            for file in file_paths:
                                fname=os.path.basename(file)
                                zip.write(file,fname)
                    else:
                        print('File already exists: '+str(filebase+'.zip')+' SKIPPING')
                except:
                    pass
                
                
                
mypath = "../../../data/Shapefile"
url_download_list_zip = []

for pagenum in range (0,100):
    # Main page url for east africa
    #url = "http://fews.net/fews-data/333?tid=23&field_data_portal_date_start%5Bvalue%5D%5Byear%5D=&field_data_portal_date_start%5Bvalue%5D%5Bmonth%5D=&field_data_portal_date_end%5Bvalue%5D%5Byear%5D=&field_data_portal_date_end%5Bvalue%5D%5Bmonth%5D=&page={0}".format(pagenum)
    url = "https://fews.net/fews-data/333?tid=34&field_data_portal_date_start%5Bvalue%5D%5Byear%5D=2009&field_data_portal_date_start%5Bvalue%5D%5Bmonth%5D=1&field_data_portal_date_end%5Bvalue%5D%5Byear%5D=2020&field_data_portal_date_end%5Bvalue%5D%5Bmonth%5D=12&page={0}".format(pagenum)

    print("Page number:" + str(pagenum))
    resp = urllib.request.urlopen(url)
    soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))
    link_list = []
    
    # Find all links from main page
    for link in soup.find_all('a', href=True): 
        link_list.append(link['href'])
        
    # if it contains node, it mains that there is data on that page number
    if any("/node/" in s for s in link_list):
        
        # we only need the content link that links to the zipfile data (shapefiles)
        url_download_list = [s for s in link_list if "/content/" in s]
        
        #Find all zip files and on that link and save that in a list
        for downloadurl in url_download_list:
            resp = urllib.request.urlopen("http://fews.net/{0}".format(downloadurl))
            soup = BeautifulSoup(resp, "lxml", from_encoding=resp.info().get_param('charset'))
            
            link_list = []
            for link in soup.find_all('a', href=True): 
                link_list.append(link['href'])
            url_download_list_zip.append([s for s in link_list if ".zip" in s][0])  
    else:
        break
        
# zip occurs twice in the extracted website link list, remove duplicates:        
url_download_list_zip = list(set(url_download_list_zip))

#DOWNLOAD FILES 
for zipfile_url in url_download_list_zip:

    filename = zipfile_url.rsplit('/', 1)[-1]

    print("Downloading shapefile {0}".format(filename))

    urllib.request.urlretrieve('{0}'.format(zipfile_url), '{0}/South_Africa_Zips/{1}'.format(mypath2, filename))
    # Extract all files into a single folder:
    archive = zipfile.ZipFile('{0}/South_Africa_Zips/{1}'.format(mypath, filename), 'r')
    archive.extractall(path="{0}/South_Africa_Assets/".format(mypath))
    
# Convert files to zips in the folder

zipshape("C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/ethiopia/foodsecurity/data/Shapefile/South_Africa_Assets",
         "C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/ethiopia/foodsecurity/data/Shapefile/South_Africa_Assets_Processed")

# order into three folders     
# CREATE FOLDERS AND MOVE FILES 

# order into three folders 
mypath2='C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/ethiopia/foodsecurity/data/Shapefile'
onlyfiles = [f for f in listdir(mypath2 + "/South_Africa_Assets_Processed/") if isfile(join(mypath2 + "/South_Africa_Assets_Processed/", f))]


try:
    shutil.rmtree(mypath2 + r"/CS") 
    shutil.rmtree(mypath2 + r"/ML1") 
    shutil.rmtree(mypath2 + r"/ML2") 
except:
    print("directory doesn't excist yet")
os.mkdir(mypath2 + r"/CS")
os.mkdir(mypath2 + r"/ML1")
os.mkdir(mypath2 + r"/ML2")

#MOVE FILES
CS = [s for s in onlyfiles if "CS" in s]

for file in CS:
    shutil.move(mypath2 + "/South_Africa_Assets_Processed/" + file, mypath2 + "/CS")

os.rename(mypath2 + r"/CS/EA201702_CS.zip", mypath2 + r"/CS/EA_201702_CS.zip")
ML1 = [s for s in onlyfiles if "ML1" in s]

for file in ML1:
    shutil.move(mypath2 + "/South_Africa_Assets_Processed/" + file, mypath2 + "/ML1")
    
ML2 = [s for s in onlyfiles if "ML2" in s]
for file in ML2:
    shutil.move(mypath2 + "/South_Africa_Assets_Processed/" + file, mypath2 + "/ML2")

# PART 2 EXTRACTION 

In [71]:

path2 ='C:\\Users\\ATeklesadik\\OneDrive - Rode Kruis\Documents\\documents\\IBF-system\\trigger-model-development\\drought\\Drought_trigger_dashboard\\'

lhz = gpd.read_file(path2+'/shapefiles/Ethiopia/ET_LHZ_2018/ET_LHZ_2018.shp')
admin = gpd.read_file(path2+'/shapefiles/Ethiopia/eth-administrative-divisions-shapefiles/eth_admbnda_adm2_csa_bofed_20201008.shp')
gdf_lhz_admin_merged = gpd.sjoin(lhz,admin,  how="right", op='intersects')
gdf_lhz_admin_merged["area"] = gdf_lhz_admin_merged['geometry'].area
idx = gdf_lhz_admin_merged.groupby(['LZCODE'])['area'].transform(max) == gdf_lhz_admin_merged['area']
df = gdf_lhz_admin_merged[idx]
df[['FNID','LZCODE','ADM2_PCODE']].to_csv(os.path.join(path2,'data/lhz_admin_ethiopia.csv'), index=False) 

C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


,LZCODE,ADM2_PCODE,area
index_right,,,
0,ET148,ET0508,3.434585
0,ET149,ET0508,3.434585
0,ET157,ET0508,3.434585
0,ET146,ET0508,3.434585
0,ET163,ET0508,3.434585
...,...,...,...
89,ET164,ET0105,1.113030
90,ET144,ET0706,0.369007
90,ET140,ET0706,0.369007


In [56]:
############### MOAMBIQUE
Country_code='SAP'

path2 ='C:\\Users\\ATeklesadik\\OneDrive - Rode Kruis\Documents\\documents\\IBF-system\\trigger-model-development\\drought\\Drought_trigger_dashboard\\'

############### 


mypath2 ='C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/IBF-system/trigger-model-development/drought/Bio_physcial_data/Raw/IPC/'


admin= gpd.read_file(path2+'/shapes/SA_admin2_region.shp')
lhz = gpd.read_file(path2+'/shapes/MZ_LHZ_2013.shp')

CS = [f for f in listdir(mypath2 + "/South_Africa_Assets_Processed/CS/") if isfile(join(mypath2 + "/South_Africa_Assets_Processed/CS/", f))]
ML1 = [f for f in listdir(mypath2 + "/South_Africa_Assets_Processed/ML1/") if isfile(join(mypath2 + "/South_Africa_Assets_Processed/ML1/", f))]
ML2 = [f for f in listdir(mypath2 + "/South_Africa_Assets_Processed/ML2/") if isfile(join(mypath2 + "/South_Africa_Assets_Processed/ML2/", f))]

df_list=list()
df_list1=list()
for file in CS:
    CS_ = gpd.read_file('zip://' +mypath2 + "/CS/" +file)
    gdf_lhz_cs_merged = gpd.sjoin(CS_,admin,  how="right", op='intersects')
    df = gdf_lhz_cs_merged.query('CS < 6')
    idx = df.groupby(['ADM2_PCODE'])['CS'].transform(max) == df['CS']
    df = df[idx]
    df = df[['ADM2_PCODE','CS']]
    x = file.split('_')[1] 
    date_item = dt.datetime(int(x[:4]),int(x[4:]),1).strftime("%Y-%m-%d")
    year_item = int(x[:4])
    df['Date'] = date_item
    df['year'] = year_item
    df_list.append(df.query('CS != "NaN"'))
    
    #############################################
    gdf_lhz_cs_merged2 = gpd.sjoin(CS_,lhz,  how="right", op='intersects')
    df2 = gdf_lhz_cs_merged2.query('CS < 6')
    idx2 = df2.groupby(['LZCODE'])['CS'].transform(max) == df2['CS']    
    df2 = df2[idx2]
    df2 = df2[['LZCODE','CS']]
    df2['Date'] = date_item
    df2['year'] = year_item
    df_list1.append(df2.query('CS != "NaN"'))
    
df1 = pd.concat(df_list)
df1.to_csv(os.path.join(path2,'data/ipc_admin_%s.csv'%Country_code), index=False)  

df_lhz = pd.concat(df_list1)
df_lhz.to_csv(os.path.join(path2,'data/ipc_lhz_%s.csv'%Country_code), index=False) 


    


C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarnin

C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarnin

In [54]:
############### KENYA
Country_code='KE'
path = "C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/ethiopia/foodsecurity/data"
path2 ='C:\\Users\\ATeklesadik\\OneDrive - Rode Kruis\Documents\\documents\\IBF-system\\trigger-model-development\\drought\\Drought_trigger_dashboard\\'

lhz = gpd.read_file(path2+'/shapefiles/kenya/KE_LHZ_2011.shp')
admin = gpd.read_file(path2+'/shapefiles/kenya/KEN_Adm2.shp')

admin =admin.rename(columns={'DSCodeAdm2':'ADM2_PCODE'}) 

mypath2='C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/ethiopia/foodsecurity/data/Shapefile/East_Africa_Assets_Processed/'

CS = [f for f in listdir(mypath2 + "/CS/") if isfile(join(mypath2 + "/CS/", f))]
ML1 = [f for f in listdir(mypath2 + "/ML1/") if isfile(join(mypath2 + "/ML1/", f))]
ML2 = [f for f in listdir(mypath2 + "/ML2/") if isfile(join(mypath2 + "/ML2/", f))]

df_list=list()
df_list1=list()
for file in CS:
    CS_ = gpd.read_file('zip://' +mypath2 + "/CS/" +file)
    gdf_lhz_cs_merged = gpd.sjoin(CS_,admin,  how="right", op='intersects')
    df = gdf_lhz_cs_merged.query('CS < 6')
    idx = df.groupby(['ADM2_PCODE'])['CS'].transform(max) == df['CS']
    df = df[idx]
    df = df[['ADM2_PCODE','CS']]
    x = file.split('_')[1] 
    date_item = dt.datetime(int(x[:4]),int(x[4:]),1).strftime("%Y-%m-%d")
    year_item = int(x[:4])
    df['Date'] = date_item
    df['year'] = year_item
    df_list.append(df.query('CS != "NaN"'))
    
    #############################################
    gdf_lhz_cs_merged2 = gpd.sjoin(CS_,lhz,  how="right", op='intersects')
    df2 = gdf_lhz_cs_merged2.query('CS < 6')
    idx2 = df2.groupby(['LZCODE'])['CS'].transform(max) == df2['CS']    
    df2 = df2[idx2]
    df2 = df2[['LZCODE','CS']]
    df2['Date'] = date_item
    df2['year'] = year_item
    df_list1.append(df2.query('CS != "NaN"'))
    
df1 = pd.concat(df_list)
df1.to_csv(os.path.join(path2,'data/ipc_admin_%s.csv'%Country_code), index=False)  

df_lhz = pd.concat(df_list1)
df_lhz.to_csv(os.path.join(path2,'data/ipc_lhz_%s.csv'%Country_code), index=False) 


    


In [48]:
############### ETHOPIA
Country_code='ETH'
path = "C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/ethiopia/foodsecurity/data"
path2 ='C:\\Users\\ATeklesadik\\OneDrive - Rode Kruis\Documents\\documents\\IBF-system\\trigger-model-development\\drought\\Drought_trigger_dashboard\\'

lhz = gpd.read_file(path2+'/shapefiles/Ethiopia/ET_LHZ_2018/ET_LHZ_2018.shp')
admin = gpd.read_file(path2+'/shapefiles/Ethiopia/eth-administrative-divisions-shapefiles/eth_admbnda_adm2_csa_bofed_20201008.shp')


mypath2='C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/ethiopia/foodsecurity/data/Shapefile/East_Africa_Assets_Processed/'

CS = [f for f in listdir(mypath2 + "/CS/") if isfile(join(mypath2 + "/CS/", f))]
ML1 = [f for f in listdir(mypath2 + "/ML1/") if isfile(join(mypath2 + "/ML1/", f))]
ML2 = [f for f in listdir(mypath2 + "/ML2/") if isfile(join(mypath2 + "/ML2/", f))]

df_list=list()
df_list1=list()
for file in CS:
    CS_ = gpd.read_file('zip://' +mypath2 + "/CS/" +file)
    gdf_lhz_cs_merged = gpd.sjoin(CS_,admin,  how="right", op='intersects')
    df = gdf_lhz_cs_merged.query('CS < 6')
    idx = df.groupby(['ADM2_PCODE'])['CS'].transform(max) == df['CS']
    df = df[idx]
    df = df[['ADM2_PCODE','CS']]
    x = file.split('_')[1] 
    date_item = dt.datetime(int(x[:4]),int(x[4:]),1).strftime("%Y-%m-%d")
    year_item = int(x[:4])
    df['Date'] = date_item
    df['year'] = year_item
    df_list.append(df.query('CS != "NaN"'))
    
    #############################################
    gdf_lhz_cs_merged2 = gpd.sjoin(CS_,lhz,  how="right", op='intersects')
    df2 = gdf_lhz_cs_merged2.query('CS < 6')
    idx2 = df2.groupby(['LZCODE'])['CS'].transform(max) == df2['CS']    
    df2 = df2[idx2]
    df2 = df2[['LZCODE','CS']]
    df2['Date'] = date_item
    df2['year'] = year_item
    df_list1.append(df2.query('CS != "NaN"'))
    
df1 = pd.concat(df_list)
df1.to_csv(os.path.join(path2,'/data/ipc_admin_%s.csv'%Country_code), index=False)  

df_lhz = pd.concat(df_list1)
df_lhz.to_csv(os.path.join(path2,'/data/ipc_lhz_%s.csv'%Country_code), index=False) 


    
 

EA_200907_CS.zip


C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


EA_200910_CS.zip


C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


EA_201001_CS.zip


...

C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


EA_201902_CS.zip


C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


EA_201906_CS.zip


C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [ ]:
pop_ETH = pd.read_csv(path + "/Shapefile/Ethiopia/Indicators_3_population.csv", sep = ";")
DATA_ETH = pd.read_csv(path + "/Processed Data/df_master_03092020.csv", sep = ";")

WOREDA['pcode']=WOREDA['WOR_P_CODE'].apply(lambda x: 'ET'+x)
WOREDA=WOREDA.merge(pop_ETH, left_on='pcode', right_on='pcode')
WOREDA.crs = {'init' :'epsg:20137'}
WOREDA = WOREDA.to_crs({'init': 'epsg:4326'})  